In [1]:
import pandas as pd
import sqlite3

С помощью sqlite3.connect создадим объект типа Connection

In [2]:
connect = sqlite3.connect('../data/checking-logs.sqlite')
connect

Создадим новую таблицу datamart в базе данных, объединив просмотры страниц таблиц и проверку с использованием одного запроса

◦ в таблице должны быть следующие столбцы: uid, labname, first_commit_ts, first_view_ts

фильтры те же:

∗ status = ’ready’

∗ numTrials = 1

◦ таблица должна содержать только пользователей (uid с user_*), а не администраторов

◦ first_commit_ts и first_view_ts должны быть проанализированы как datetime64[ns]

In [3]:
request = '''
SELECT checker.uid,
       checker.labname,
       checker.timestamp AS first_commit_ts,
       pageviews.datetime AS first_view_ts
FROM checker 
LEFT JOIN pageviews ON checker.uid=pageviews.uid
WHERE status='ready'
AND numTrials=1
AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND checker.uid LIKE 'user_%'
AND (pageviews.datetime = (SELECT MIN(pageviews.datetime)
                           FROM pageviews
                           WHERE uid=checker.uid)
        OR pageviews.datetime IS NULL)
'''
datamart = pd.io.sql.read_sql(request, connect, parse_dates=[
                              'first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


Создадим два фрейма данных: тестовый и контрольный:

у тестового должны быть пользователи, у которых есть значения в first_view_ts

у контрольного должны быть пользователи, у которых отсутствуют значения в first_view_ts

In [4]:
test = datamart[datamart['first_view_ts'].notnull()]
control = datamart[datamart['first_view_ts'].isnull()]

Найдем среднее значения first_view_ts

In [5]:
replase_arg = test['first_view_ts'].mean()
replase_arg

Timestamp('2020-04-27 00:40:05.761783552')

Заменим недостающие значения на средние

In [6]:
control = control.fillna(test['first_view_ts'].mean())

Сохраним обе таблицы в базе данных

In [7]:
test.to_sql('test', connect)
control.to_sql('control', connect)

Закроем соединение с базой данных

In [8]:
connect.close()